In [2]:


import os
import json
import requests
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from openai import OpenAI
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import random
from datetime import datetime

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class InterparkTicketCrawler:
    def __init__(self, creds='google.json', sheet_name='감사한 티켓팅 신청서'):
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(creds, scope)
        self.sheet = gspread.authorize(creds).open(sheet_name).worksheet('Hot')

        # 캐시 파일 경로
        self.artist_cache_path = Path('artist_cache.json')
        self.hashtag_cache_path = Path('hashtag_cache.json')
        self.tweet_cache_path = Path('tweet_cache.json')

        # 캐시 로딩
        self.artist_cache = self.load_cache(self.artist_cache_path)
        self.hashtag_cache = self.load_cache(self.hashtag_cache_path)
        self.tweet_cache = self.load_cache(self.tweet_cache_path)
        
    def load_cache(self, path: Path) -> dict:
        if path.exists():
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                pass
        return {}

    def save_cache(self, cache: dict, path: Path):
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(cache, f, ensure_ascii=False, indent=2)

    def fetch_data(self):
        url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"
        params = {"goodsGenre": "ALL", "goodsRegion": "ALL", "offset": 0, "pageSize": 400, "sorting": "OPEN_ASC"}
        headers = {
            "user-agent": "Mozilla/5.0",
            "referer": "https://tickets.interpark.com/contents/notice"
        }
        r = requests.get(url, params=params, headers=headers)
        r.raise_for_status()
        return r.json()

    def filter_hot(self, data):
        hot = []
        for d in data:
            if d.get('goodsGenreStr') == '뮤지컬' and not d.get('isHot'):
                continue
            if d.get('goodsGenreStr') != '뮤지컬' and d.get('viewCount', 0) <= 1000:
                continue
            hot.append({
                '오픈시간': d.get('openDateStr', ''),
                '조회수': d.get('viewCount', 0),
                '예매타입': d.get('openTypeStr', ''),
                '제목': d.get('title', ''),
                '예매코드': d.get('goodsCode', ''),
                '장르': d.get('goodsGenreStr', ''),
                'Image': d.get('posterImageUrl', '')
            })
        return hot

    def extract_artist(self, title: str) -> str:
        if title in self.artist_cache:
            return self.artist_cache[title]

        prompt = f"""
아래는 콘서트 제목이야. 여기서 가수명이나 그룹명만 간단히 추출해줘. 뮤지컬일 경우 뮤지컬 제목만 추출해줘.**영문일 경우 한글도 같이 작성해야되고, 약어가 있으면 풀네임이랑 약어도 같이 작성해야해**
예시: 악동뮤지션 (악뮤, AKMU)
제목: {title}
가수명 or 뮤지컬 제목:"""

        try:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
            )
            artist = res.choices[0].message.content.strip().strip('"')
            self.artist_cache[title] = artist
            return artist
        except Exception as e:
            print(f"❌ OpenAI 오류 (가수명): {e}")
            return "불명"

    def generate_hashtags(self, title: str, artist: str, genre: str) -> str:
        key = f"{title}"
        if key in self.hashtag_cache:
            return self.hashtag_cache[key]

        prompt = f"""
콘서트 제목: {title}
가수 또는 뮤지컬 제목: {artist}
장르: {genre}

위 콘서트를 대리티켓팅 목적으로 트위터에 해시태그 10개를 한국어로 작성해줘.
형식: #블랙핑크콘서트 #블랙핑크 #BLACKPINK #블핑댈티 #대리티켓팅
조건: '#' 포함하고 띄어쓰기 없이, 한 줄로 콤마 없이 출력해줘.
"""

        try:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
            )
            hashtags = res.choices[0].message.content.strip()
            self.hashtag_cache[key] = hashtags
            return hashtags
        except Exception as e:
            print(f"❌ OpenAI 오류 (해시태그): {e}")
            return "#대리티켓팅"

    def add_ai_columns(self, df):
        print("🤖 가수명 + 해시태그 생성 중...")
        artists = []
        hashtags = []

        for _, row in tqdm(df.iterrows(), total=len(df)):
            title = row['제목']
            genre = row['장르']

            artist = self.extract_artist(title)
            hashtag = self.generate_hashtags(title, artist, genre)

            artists.append(artist)
            hashtags.append(hashtag)

        df['가수명'] = artists
        df['해시태그'] = hashtags

        self.save_cache(self.artist_cache, self.artist_cache_path)
        self.save_cache(self.hashtag_cache, self.hashtag_cache_path)

        return df
    
    def add_twitter_columns(self, df):
        print("🤖 트위터 문구 생성 중...")
        with open('tweet_templates.json', 'r', encoding='utf-8') as f:
            templates = json.load(f)
        
        tweet_contents = []
        for _, row in tqdm(df.iterrows(), total=len(df)):
            if row['조회수'] > 10000:
                template = random.choice(templates)
            else:
                template = {"content": "{title}\n\n🚨 {singer} 대리티켓팅(댈티)\n티켓오픈시간: {open_time}\n상담 링크: https://open.kakao.com/o/sAJ8m2Ah\n\n{hash_tag} #평생한번 #놓치면후회 #앞열보장"}
            
            # 시간 치환
            
            title = row['제목']
            singer = row['가수명']
            
            # 오픈시간이 문자열인 경우 datetime으로 변환
            open_time_raw = row['오픈시간']
            if isinstance(open_time_raw, str):
                # 문자열을 datetime으로 변환
                open_time_dt = datetime.strptime(open_time_raw, '%Y-%m-%d %H:%M:%S')
                open_time = open_time_dt.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            else:
                # 이미 datetime 객체인 경우
                open_time = open_time_raw.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            
            hash_tag = row['해시태그']
            content = template['content'].replace("{open_time}", open_time).replace("{title}", title).replace("{singer}", singer).replace("{hash_tag}", hash_tag)
            tweet_contents.append(content)

        df['트위터'] = tweet_contents
        self.save_cache(self.tweet_cache, self.tweet_cache_path)
        return df
        

    def update_sheet(self, df):
        self.sheet.clear()
        if df.empty:
            print("📭 HOT 티켓 없음")
            return
        self.sheet.append_row(list(df.columns))
        for row in df.values.tolist():
            self.sheet.append_row(row)
        print(f"✅ {len(df)}개 티켓 업로드 완료")

    def run(self):
        raw = self.fetch_data()
        hot = self.filter_hot(raw)
        df = pd.DataFrame(hot)
        if df.empty:
            return df
        df = df.sort_values(by='오픈시간')
        df = self.add_ai_columns(df)
        df = self.add_twitter_columns(df)
        self.update_sheet(df)
        return df

if __name__ == "__main__":
    df = InterparkTicketCrawler().run()
    if not df.empty:
        print("\n📋 HOT 티켓 요약:")
        print(df[['오픈시간', '제목', '가수명', '해시태그']].to_string(index=False))

🤖 가수명 + 해시태그 생성 중...


100%|██████████| 10/10 [00:00<00:00, 7986.11it/s]


🤖 트위터 문구 생성 중...


100%|██████████| 10/10 [00:00<00:00, 6891.73it/s]


✅ 10개 티켓 업로드 완료

📋 HOT 티켓 요약:
               오픈시간                                                                    제목                                  가수명                                                                                      해시태그
2025-07-14 20:00:00                                       2025 영탁 단독 콘서트 "TAK SHOW4" - 전주                                   영탁                      #영탁콘서트 #영탁 #TAKSHOW4 #전주콘서트 #영탁대리티켓팅 #대리티켓팅 #영탁팬 #콘서트티켓팅 #전주공연 #영탁공연
2025-07-15 20:00:00                                     2025 AKMU STANDING CONCERT ［악동들］                      악동뮤지션 (악뮤, AKMU)                   #악뮤콘서트 #악동뮤지션 #AKMU #악뮤 #악뮤대리티켓팅 #악뮤콘서트2025 #악동들콘서트 #대리티켓팅 #악뮤팬 #콘서트티켓팅
2025-07-15 20:00:00                                      뮤 내한공연 (MEW The Farewell Shows)                               뮤 (MEW)                          #뮤내한공연 #뮤콘서트 #뮤티켓팅 #뮤대리티켓팅 #뮤팬 #뮤음악 #콘서트대리 #티켓팅대행 #뮤직페스티벌 #뮤직라이브
2025-07-16 14:00:00                                                          뮤지컬 〈맘마미아!〉              

In [3]:
import os
import tweepy
import requests
from dotenv import load_dotenv
import time
import random
from tqdm import tqdm

load_dotenv()

class PostTweet:
    def __init__(self):
        auth = tweepy.OAuthHandler(os.getenv('TWITTER_API_KEY'), os.getenv('TWITTER_API_SECRET'))
        auth.set_access_token(os.getenv('TWITTER_ACCESS_TOKEN'), os.getenv('TWITTER_ACCESS_TOKEN_SECRET'))
        self.api = tweepy.API(auth)
        self.client = tweepy.Client(
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )

    def _download_image(self, url):
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            path = f"image/{url.split('/')[-1]}"
            with open(path, "wb") as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)
            return path
        return None

    def post(self, text, image_url=None):
        media_ids = []
        if image_url:
            path = self._download_image(image_url)
            if path:
                media = self.api.media_upload(path)
                media_ids.append(media.media_id)

        tweet = self.client.create_tweet(text=text, media_ids=media_ids if media_ids else None)
        print(f"https://twitter.com/gamsahanticket/status/{tweet.data['id']}")


for _, row in tqdm(df.iterrows(), total=len(df)):
    title = row['제목']
    text = row['트위터']
    image_url = row['Image']
    PostTweet().post(text, image_url)
    print(f"🔄 {title} 트윗 게시 완료")
    sleep_time = random.randint(60, 90)
    
    # 실시간 카운트다운
    for remaining in range(sleep_time, 0, -1):
        print(f"\r⏰ 다음 트윗까지 {remaining}초 남음...", end="", flush=True)
        time.sleep(1)
    
    print(f"\n🔄 {sleep_time}초 대기 완료, 다음 트윗 게시")


  0%|          | 0/10 [00:00<?, ?it/s]

https://twitter.com/gamsahanticket/status/1943957815670059515
🔄 2025 영탁 단독 콘서트 "TAK SHOW4" - 전주 트윗 게시 완료


 10%|█         | 1/10 [01:12<10:55, 72.83s/it]

🔄 71초 대기 후 다음 트윗 게시
https://twitter.com/gamsahanticket/status/1943958122655297983
🔄 2025 AKMU STANDING CONCERT ［악동들］  트윗 게시 완료


 20%|██        | 2/10 [02:40<10:50, 81.29s/it]

...

🔄 82초 대기 후 다음 트윗 게시
https://twitter.com/gamsahanticket/status/1943960405489201364
🔄 민트페스타 vol.78 SPIRITED 트윗 게시 완료


 90%|█████████ | 9/10 [11:38<01:19, 79.42s/it]

🔄 79초 대기 후 다음 트윗 게시
https://twitter.com/gamsahanticket/status/1943960743269085360
🔄 TOMORROW X TOGETHER WORLD TOUR 〈ACT : TOMORROW〉  IN SEOUL  트윗 게시 완료


100%|██████████| 10/10 [13:09<00:00, 78.98s/it]

🔄 90초 대기 후 다음 트윗 게시
